# Benchmarks

## Initialize

In [1]:
import os
import math
import pathlib
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.feather as feather
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [2]:
base_path = "/home/jakobs"

project_path = f"{base_path}/data"

experiment = '230323'
experiment_path = f"{project_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

partitions = [i for i in range(10)]

today = experiment

In [3]:
endpoint_defs = pd.read_feather(f"{base_path}/data/endpoints_epic_md.feather").set_index("endpoint")
endpoints = endpoint_defs.index.to_list()

In [4]:
out_path = f"{experiment_path}/coxph/predictions"
pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)

In [5]:
from sklearn.preprocessing import StandardScaler
import pickle
import zstandard

def read_data(fp_in):
    temp = pd.read_feather(f"{fp_in}").set_index("eid")
    return temp   
    
def save_pickle(data, data_path):
    with open(data_path, "wb") as fh:
        cctx = zstandard.ZstdCompressor()
        with cctx.stream_writer(fh) as compressor:
            compressor.write(pickle.dumps(data, protocol=pickle.HIGHEST_PROTOCOL))
    
def read_predictions(endpoint, feature_set, partition, model):
    
    #identifier = f"{endpoint}_{feature_set}_{model}_{partition}_mean" # for meaned preds
    identifier = f"{endpoint}_{feature_set}_{model}_{partition}"
    fp_in = f"{out_path}/{identifier}.feather"
    
    temp = read_data(fp_in)
    return temp

In [6]:
model_path = pathlib.Path(f"{experiment_path}/coxph/input")
models = [f.name for f in model_path.iterdir() if f.is_dir() and "ipynb_checkpoints" not in str(f)]
models

['RetinaUKB']

In [7]:
d = []

for i, endpoint in enumerate(tqdm(endpoints)):
    for feature_set in [
        "Age+Sex",
        "Retina",
        "Age+Sex+Retina",
         "SCORE2",
         "SCORE2+Retina",
         "ASCVD",
         "ASCVD+Retina",
#         "QRISK3",
#         "QRISK3+Retina"
                    ]:
        for partition in partitions:
            for model in models:
                try: 
                    temp = read_predictions(endpoint, feature_set, partition, model)
                    d.append({"endpoint": endpoint, "features":feature_set, "model":model, "partition":partition, "available": True})
                except:
                    d.append({"endpoint": endpoint, "features":feature_set, "partition":partition, "available": False})

  0%|          | 0/368 [00:00<?, ?it/s]

In [8]:
pd.DataFrame.from_dict(d)['available'].unique()

array([ True])

In [9]:
pd.DataFrame.from_dict(d).groupby(["features"])["available"].sum().to_frame()

,available
features,
ASCVD,3680
ASCVD+Retina,3680
Age+Sex,3680
Age+Sex+Retina,3680
Retina,3680
SCORE2,3680
SCORE2+Retina,3680


In [10]:
pd.DataFrame.from_dict(d).groupby(["model", "features"])["available"].sum().to_frame()

available
model     features                 
RetinaUKB ASCVD                3680
          ASCVD+Retina         3680
          Age+Sex              3680
          Age+Sex+Retina       3680
          Retina               3680
          SCORE2               3680
          SCORE2+Retina        3680

In [11]:
in_path = f"{experiment_path}/coxph/predictions"
prediction_paths = !ls $in_path
print(prediction_paths[0])
predictions = pd.Series(prediction_paths).str.split("_", expand=True)\
    .assign(path = prediction_paths)\
    .assign(endpoint = lambda x: x[0]+"_"+x[1])\
    .assign(score = lambda x: x[2])\
    .assign(model = lambda x: x[3])\
    .assign(partition = lambda x: x[4].str.replace(".feather", "", regex=True).astype(int))\
    [["model", "endpoint", "score", "partition", "path"]].sort_values(["model", "endpoint", "score", "partition"]).reset_index(drop=True)
predictions

OMOP_4306655_ASCVD+Retina_RetinaUKB_0.feather


,model,endpoint,score,partition,path
0,RetinaUKB,OMOP_4306655,ASCVD,0,OMOP_4306655_ASCVD_RetinaUKB_0.feather
1,RetinaUKB,OMOP_4306655,ASCVD,1,OMOP_4306655_ASCVD_RetinaUKB_1.feather
2,RetinaUKB,OMOP_4306655,ASCVD,2,OMOP_4306655_ASCVD_RetinaUKB_2.feather
3,RetinaUKB,OMOP_4306655,ASCVD,3,OMOP_4306655_ASCVD_RetinaUKB_3.feather
4,RetinaUKB,OMOP_4306655,ASCVD,4,OMOP_4306655_ASCVD_RetinaUKB_4.feather
...,...,...,...,...,...
25755,RetinaUKB,phecode_979,SCORE2+Retina,5,phecode_979_SCORE2+Retina_RetinaUKB_5.feather
25756,RetinaUKB,phecode_979,SCORE2+Retina,6,phecode_979_SCORE2+Retina_RetinaUKB_6.feather
25757,RetinaUKB,phecode_979,SCORE2+Retina,7,phecode_979_SCORE2+Retina_RetinaUKB_7.feather
25758,RetinaUKB,phecode_979,SCORE2+Retina,8,phecode_979_SCORE2+Retina_RetinaUKB_8.feather


In [12]:
predictions.to_feather(f"{experiment_path}/prediction_paths.feather")